In [1]:
import sys
sys.path.append("..")
from pathlib import Path
import numpy as np
from scipy import stats
import importlib
import src.assay_calibration.fit_utils.two_sample.fit
from src.assay_calibration.fit_utils.fit import Fit
importlib.reload(src.assay_calibration.fit_utils.two_sample.fit)
importlib.reload(src.assay_calibration.fit_utils.fit)
from src.assay_calibration.fit_utils.two_sample.fit import single_fit
from src.assay_calibration.fit_utils.two_sample import (density_utils,constraints, optimize)
import scipy.stats as sps
import matplotlib
matplotlib.set_loglevel("warning")
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import trange
import os
sys.path.append(str(Path(os.getcwd()).parent))
from src.assay_calibration.data_utils.dataset import (
    PillarProjectDataframe,
    Scoreset,
    BasicScoreset,
)
import json
import glob

In [2]:
def test_fit(ds, component_range, check_monotonic, init_strategy):
    # Load directly from JSON
    fit = Fit(ds)
    fits = fit.run(
                core_limit=20,
                num_fits=20,
                verbose_level=20,
                component_range=component_range,
                bootstrap=False,
                verbose=False,
                max_em_iters=10000,
                check_convergence=False,
                check_monotonic=check_monotonic,
                init_strategy=init_strategy,
                score_min=ds.scores.min() - 1,
                score_max=ds.scores.max() + 1,
                # kmean_init="k-means++"
            )
    
    return fits, ds

# for dataset_f in glob.glob("/data/ross/assay_calibration/scoresets/*.json"):
    
#     dataset_name = dataset_f.split('/')[-1][:-5]

#     for component_range in ("2-component","3-component"):
#         for monotonicity_contraint in ("no constraint","constraint"):
#             # for init_strategy in ("kmeans","random"):
#             init_strategy = "kmeans"
            
#             fits, ds = test_fit(dataset_f, component_range=[3] if component_range[0] == "3" else [2], 
#                                 check_monotonic=False if monotonicity_contraint[0] == "n" else True, 
#                                 init_strategy=init_strategy)
            
#             fit_results = sorted(fits, key=lambda res: res['likelihoods'][-1], reverse=True)
#             best_fit = fit_results[0]
#             scores = ds.scores
#             sample_assignments = ds.sample_assignments
            
#             score_range = np.linspace(scores.min(), scores.max(), 1000)
#             estimatedDensities = np.array([density_utils.joint_densities(score_range[...,None],
#                                                                          best_fit['component_params'],
#                                                                          sample_weights).squeeze() for sample_weights in best_fit['weights']])
            
#             fig,ax = plt.subplots(3,1, figsize=(12,12))
#             for i in range(3):
#                 ax[i].plot(score_range, estimatedDensities[i].sum(0), label='Estimated', color='C1', linestyle='--')
#                 ax[i].hist(scores[sample_assignments[:,i]], density=True, alpha=0.3, color='gray', label='Data histogram')
#                 ax[i].set_title(f'Sample {i+1}')
#                 ax[i].legend()

In [ ]:
#### import matplotlib.pyplot as plt
import glob
import os
import numpy as np

# Create output directory if it doesn't exist
output_dir = "/data/ross/assay_calibration/test_experimental_plots_v3"
os.makedirs(output_dir, exist_ok=True)

# Suppress matplotlib debug messages
import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)

for dataset_f in glob.glob("/data/ross/assay_calibration/scoresets/*.json"):
    
    dataset_name = dataset_f.split('/')[-1][:-5]
    print(f"Processing {dataset_name}")

    ds = Scoreset.from_json(dataset_f)
    print(ds)
    
    # Create figure with 4x3 subplot grid
    fig, axes = plt.subplots(4, 3, figsize=(18, 20))
    fig.suptitle(f'Dataset: {dataset_name}', fontsize=16, y=0.995)
    
    plot_idx = 0
    for component_range in ("2-component", "3-component"):
        for monotonicity_constraint in ("no constraint", "constraint"):
            init_strategy = "random"
            
            # Run the fit
            try:
                fits, ds = test_fit(
                    ds, 
                    component_range=[3] if component_range[0] == "3" else [2], 
                    check_monotonic=False if monotonicity_constraint[0] == "n" else True, 
                    init_strategy=init_strategy
                )
                
                # Get best fit
                fit_results = sorted(fits, key=lambda res: res['likelihoods'][-1], reverse=True)
                best_fit = fit_results[0]
                best_init = "MoM" if best_fit['kmeans'] == "method_of_moments" else "KM"
                scores = ds.scores
                sample_assignments = ds.sample_assignments
                
                # Calculate densities
                score_range = np.linspace(scores.min(), scores.max(), 1000)
                estimatedDensities = np.array([
                    density_utils.joint_densities(
                        score_range[..., None],
                        best_fit['component_params'],
                        sample_weights
                    ).squeeze() for sample_weights in best_fit['weights']
                ])
                
                # Plot for each sample (3 columns)
                for i in range(3):
                    ax = axes[plot_idx, i]
                    
                    # Plot estimated density
                    ax.plot(score_range, estimatedDensities[i].sum(0), 
                           label='Estimated', color='C1', linestyle='-', linewidth=2)
                    
                    # Plot histogram of actual data
                    if i < sample_assignments.shape[1] and sample_assignments[:, i].sum() > 0:
                        ax.hist(scores[sample_assignments[:, i]], 
                               bins=30, density=True, alpha=0.3, 
                               color='gray', label='Data')
                    
                    # Set labels and title
                    if i == 0:
                        ax.set_ylabel(f'{component_range}, {monotonicity_constraint}, {best_init}', 
                                     fontsize=10, fontweight='bold')
                    
                    if plot_idx == 0:
                        ax.set_title(f'Sample {i+1}', fontsize=12)
                    
                    if plot_idx == 3:
                        ax.set_xlabel('Score', fontsize=10)
                    
                    ax.legend(loc='upper right', fontsize=8)
                    ax.grid(True, alpha=0.2)
                    
                    # Add likelihood value as text
                    likelihood = best_fit['likelihoods'][-1]
                    ax.text(0.02, 0.98, f'LL: {likelihood:.3f}', 
                           transform=ax.transAxes, fontsize=8,
                           verticalalignment='top')
                
            except Exception as e:
                print(f"  Error with {component_range}, {monotonicity_constraint}: {e}")
                # Create empty plots with error message
                for i in range(3):
                    ax = axes[plot_idx, i]
                    ax.text(0.5, 0.5, f'Error:\n{str(e)[:30]}...', 
                           ha='center', va='center', transform=ax.transAxes)
                    ax.set_xticks([])
                    ax.set_yticks([])
                    
                    if i == 0:
                        ax.set_ylabel(f'{component_range}, {monotonicity_constraint}, {best_init}', 
                                     fontsize=10, fontweight='bold')
                    if plot_idx == 0:
                        ax.set_title(f'Sample {i+1}', fontsize=12)
            
            plot_idx += 1
    
    # Adjust layout and save
    plt.tight_layout(rect=[0, 0, 1, 0.99])
    
    output_path = f"{output_dir}/{dataset_name}.png"
    plt.savefig(output_path, dpi=100, bbox_inches='tight')
    plt.close()
    
    print(f"  Saved to {output_path}")

print("All plots saved!")

Processing ASPA_Grønbæk-Thygesen_2024_abundance
ASPA_Grønbæk-Thygesen_2024_abundance: 18623 total variants
	Pathogenic/Likely Pathogenic: 64 variants
	Benign/Likely Benign: 6 variants
	gnomAD: 352 variants



  Saved to /data/ross/assay_calibration/test_experimental_plots_v3/ASPA_Grønbæk-Thygesen_2024_abundance.png
Processing ASPA_Grønbæk-Thygesen_2024_toxicity
ASPA_Grønbæk-Thygesen_2024_toxicity: 18615 total variants
	Pathogenic/Likely Pathogenic: 64 variants
	Benign/Likely Benign: 6 variants
	gnomAD: 352 variants



  Saved to /data/ross/assay_calibration/test_experimental_plots_v3/ASPA_Grønbæk-Thygesen_2024_toxicity.png
Processing BAP1_Waters_2024
BAP1_Waters_2024: 18108 total variants
	Pathogenic/Likely Pathogenic: 197 variants
	Benign/Likely Benign: 398 variants
	gnomAD: 1369 variants
	Synonymous: 1854 variants



  Saved to /data/ross/assay_calibration/test_experimental_plots_v3/BAP1_Waters_2024.png
Processing BARD1_unpublished
BARD1_unpublished: 9128 total variants
	Pathogenic/Likely Pathogenic: 185 variants
	Benign/Likely Benign: 270 variants
	gnomAD: 1614 variants
	Synonymous: 1337 variants



  Saved to /data/ross/assay_calibration/test_experimental_plots_v3/BARD1_unpublished.png
Processing BRCA1_Adamovich_2022_Cisplatin
BRCA1_Adamovich_2022_Cisplatin: 5131 total variants
	Pathogenic/Likely Pathogenic: 55 variants
	Benign/Likely Benign: 25 variants
	gnomAD: 115 variants



...

EM Iteration:  20%|█▉        | 1951/10000 [00:14<01:00, 132.55it/s, likelihood=0.087004]1]

  Error with 3-component, no constraint: Likelihood decreased at iteration 1777 for unconstrained fit


  Saved to /data/ross/assay_calibration/test_experimental_plots_v3/BRCA2_Sahu_2023_exon13_SGE.png
Processing BRCA2_Sahu_2023_exon13_global_score
BRCA2_Sahu_2023_exon13_global_score: 251 total variants
	Pathogenic/Likely Pathogenic: 17 variants
	Benign/Likely Benign: 9 variants
	gnomAD: 46 variants
	Synonymous: 39 variants



  Saved to /data/ross/assay_calibration/test_experimental_plots_v3/BRCA2_Sahu_2023_exon13_global_score.png
Processing BRCA2_Sahu_2025_HDR
BRCA2_Sahu_2025_HDR: 6550 total variants
	Pathogenic/Likely Pathogenic: 353 variants
	Benign/Likely Benign: 71 variants
	gnomAD: 938 variants
	Synonymous: 1276 variants



EM Iteration:  14%|█▍        | 1424/10000 [00:17<01:44, 82.17it/s, likelihood=-0.879044] 

In [ ]:
constraints.multicomponent_density_constraint_violated(best_fit['component_params'],
                                                       score_range[[0,-1]])

In [ ]:
best_fit.keys()

In [ ]:
best_fit['component_params']

In [ ]:
scores.min(),scores.max()